## Imports

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
from pylab import rcParams

import warnings
warnings.filterwarnings('ignore')

In [11]:
orders = pd.read_csv('orders.csv', sep=';')

In [ ]:
items = pd.read_csv('items.csv', sep=';')

## Data Cleaning

In [ ]:
# drop null payment rows
payfilter = orders['payment'].isna()
orders.drop(orders[payfilter].index, inplace=True)

# drop items that are not sold by Top4Sport
techfilter = items['group0_id'].isna()
items.drop(items[techfilter].index, inplace=True)

In [ ]:
# create columns used for visualizations
orders['date']=pd.to_datetime(orders['date'])
orders['net_revenue'] = orders['quantity'] * orders['unit_price_vat_excl']
orders['gross_revenue'] = orders['quantity'] * orders['unit_rrp_vat_excl']
orders['net_profit'] = orders['quantity'] * (orders['unit_price_vat_excl'] - orders['unit_cogs'])

In [ ]:
# drop 'other' sites because almost no information loss
orders.drop(orders[orders['site']=='other'].index, inplace=True)
# extract site country domain and create new country column
orders['country']=orders['site'].apply(lambda x: x.split('.')[1])
# drop .com domains because almost no information loss and company based in Europe
orders.drop(orders[orders['country']=='com'].index, inplace=True)
# map domain to country
orders['country']=orders['country'].replace(
    {'cz':'Czech Republic', 'de':'Germany', 'es':'Spain', 'hr': 'Croatia',  'it': 'Italy',
     'fr':'France',  'hu':'Hungary', 'at':'Austria', 'ie':'Ireland', 'ro':'Romania', 'sk':'Slovakia',
     'dk':'Denmark', 'nl':'Netherlands', 'se':'Sweden', 'pt':'Portugal', 'pl':'Poland',
     'be':'Belgium', 'fi':'Finland', 'si':'Slovenia', 'uk':'United Kingdom', 'bg':'Bulgaria'
})
# drop unnecessary order columns
order_df=orders.drop(['payment','unit_rrp_vat_excl', 'department', 'channel', 'owner', 'site', 'CreatedAt', 'gift_quantity'], axis=1)
# drop unnecessary item columns
items_df=items.drop(['brand_id','group0_id', 'group1_id', 'group2_id'], axis=1)

In [ ]:
viz_df=pd.merge(order_df, items_df, how='left', on='item_code')

In [ ]:
viz_df.to_csv('data_cleaned.csv', index=False)

In [ ]:
viz_df.date.dtype